In [1]:
import pandas as pd

import numpy as np
np.random.seed(0)

from IPython.display import display
import ipywidgets as widgets

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_validate, cross_val_predict, learning_curve
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error
from scikitplot.estimators import plot_learning_curve
from functools import partial
from catboost import CatBoostRegressor
import xgboost as xgb
import re
import sys

import scikitplot as skplt

import brewer2mpl
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

%matplotlib inline

In [2]:
# Załadowanie danych do modelu

In [3]:
train = pd.read_hdf('../input/train.car_price.h5') # Zbiór treningowy
test = pd.read_hdf('../input/test.car_price.h5') # Zbiór testowy bez odpowiedzi

In [4]:
# Połączenie ze sobą zbiorów treningowego oraz testowego.
# Wypełnienie wszystkich braków danych wartością -1.

In [5]:
df_all = pd.concat([train, test], axis=0)

In [6]:
df_all = df_all.fillna(-1)

In [7]:
df_all.sample(5)

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,...,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
63668,"[Osobowe, Opel, Insignia]","07:54, 4 marca 2018",PLN,Cena Brutto,37900.0,"GROTA ROWECKIEGO 10 - 30-348 Kraków, Łagiewnik...",F.H. AUTO HIT Tomasz Wójcikiewicz,Dealer,False,False,...,-1,Używane,-1,-1,-1,-1,Tak,-1,-1,63668
85811,"[Osobowe, Volkswagen, CC]","12:55, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji",64500.0,"Sieradz, sieradzki, Łódzkie",-1,Osoba prywatna,True,False,...,-1,Używane,-1,-1,-1,-1,-1,-1,-1,85811
127515,"[Osobowe, BMW, Seria 2]","16:40, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",99000.0,"Łęczna, łęczyński, Lubelskie",-1,Osoba prywatna,True,True,...,-1,Używane,-1,-1,-1,-1,-1,-1,-1,127515
137658,"[Osobowe, Ford, Fiesta, Ford Fiesta Mk7 (2008-)]","09:19, 5 marca 2018",PLN,-1,-1.0,"Szczytno, szczycieński, Warmińsko-mazurskie",-1,Osoba prywatna,False,True,...,-1,Używane,Mk7 (2008-),-1,-1,-1,Tak,-1,-1,137658
89762,"[Osobowe, Audi, A3, Audi A3 8V (2012-)]","17:25, 4 marca 2018",PLN,-1,-1.0,"Zelów, bełchatowski, Łódzkie",-1,Osoba prywatna,True,False,...,-1,Używane,8V (2012-),-1,-1,-1,-1,-1,-1,89762


In [8]:
# Usunięcie pierwszych sześciu kolumn. Choć zawierają one sporo ciekawych informacji na początek nie ma potrzeby wyciągania
# dodatkowych cech z poszczególnych kolumn.

In [9]:
if 'breadcrumb' in df_all: del df_all['breadcrumb']
if 'created_at' in df_all: del df_all['created_at']
if 'seller_address' in df_all: del df_all['seller_address']
if 'price_details' in df_all: del df_all['price_details']
if 'seller_name' in df_all: del df_all['seller_name']
if 'price_currency' in df_all: del df_all['price_currency']

In [10]:
# Cześć danych dot. samochodów zapisana jest dwóch odpowiadających sobie nazwom kolumnach (POL, ENG).
# Poniżej funkcja scalająca równoważne kolumny w jedną. Dzięki temu część braków danych zostanie uzupełniona.

In [11]:
def param(df, c1, c2):
    for i in range (1, df.shape[0]):
        if df.loc[i, c1] != -1:
            return df.loc[i, c1] == df.loc[i, c1]
        else:
            return df.loc[i, c1] == df.loc[i, c2]

In [12]:
param(df_all, 'param_liczba-miejsc', 'param_nr-of-seats')
param(df_all, 'param_liczba-drzwi', 'param_door-count')
param(df_all, 'param_rok-produkcji', 'param_year')
param(df_all, 'param_przebieg', 'param_mileage')
param(df_all, 'param_pojemność-skokowa', 'param_engine-capacity')
param(df_all, 'param_napęd', 'param_transmission ')
param(df_all, 'param_rodzaj-paliwa', 'param_fuel-type')
param(df_all, 'param_typ', 'param_body-type')
param(df_all, 'param_wersja', 'param_version')
param(df_all, 'param_oferta-od', 'seller_type')
param(df_all, 'param_kraj-pochodzenia', 'param_country-of-origin')
param(df_all, 'param_skrzynia-biegów', 'param_gearbox')
param(df_all, 'param_kolor', 'param_color')
param(df_all, 'param_uszkodzony', 'param_damaged')
param(df_all, 'param_perłowy', 'param_pearl')
param(df_all, 'param_metalik', 'param_metallic')
param(df_all, 'param_pierwszy-właściciel', 'param_original-owner')
param(df_all, 'param_zarejestrowany-w-polsce', 'param_registered-in-poland')
param(df_all, 'param_bezwypadkowy', 'param_no-accident')
param(df_all, 'param_filtr-cząstek-stałych', 'param_particle-filter')
param(df_all, 'param_leasing', 'param_leasing-concession')
param(df_all, 'param_akryl-(niemetalizowany)', 'param_acrylic')
param(df_all, 'param_serwisowany-w-aso', 'param_service-record')
param(df_all, 'param_marka-pojazdu', 'param_make')
param(df_all, 'param_model-pojazdu', 'param_model')
param(df_all, 'param_rodzaj-paliwa', 'param_fuel-type')
param(df_all, 'param_moc', 'param_engine-power')

True

In [13]:
# Dane zawierają cechy o nazwie feature np. klimatyzacja, elektryczne lusterka etc. Typ danych - bool.
# Poniżej pętla konwertująca cechy bool na categorical oraz utworzenie cechy 'amenities', czyli udogodnienia, bo zakładamy,
# że im więcej udogodnień tym droższy samochód.

In [14]:
cat_feats = df_all.select_dtypes(include=[np.bool]).columns

for cat_feat in cat_feats:
    df_all['{0}_cat'.format(cat_feat)] = df_all[cat_feat].factorize()[0]

In [15]:
f_list = df_all[df_all.columns[pd.Series(df_all.columns).str.contains("feature" and "cat")]]

In [16]:
amenities = f_list.sum(axis = 1)
df_all['amenities'] = amenities

In [17]:
# Zbiór danych zawiera cechy, które mają wiele różnych klas w sobie.
# Klasy te można pogrupować na podstawie np. wykresów cena vs. cecha.

In [18]:
top_drzwi = list(df_all['param_liczba-drzwi'].value_counts().head(5).keys())
drzwi_norm = defaultdict(lambda: 'Other', dict(zip(top_drzwi, top_drzwi)))
df_all['drzwi_norm'] = df_all['param_liczba-drzwi'].map(drzwi_norm)

In [19]:
top_kraj = list(df_all['param_kraj-pochodzenia'].value_counts().head(12).keys())
kraj_norm = defaultdict(lambda: 'Inny', dict(zip(top_kraj, top_kraj)))
df_all['kraj_norm'] = df_all['param_kraj-pochodzenia'].map(kraj_norm)

In [20]:
df_all['param_metalik'] = df_all['param_metalik'].replace(['metallic'],'Tak')

In [21]:
df_all['param_typ'] = df_all['param_typ'].replace(['not enough details'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['sedan/coupe/cabrio'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['hatchback'],'Kompakt')
df_all['param_typ'] = df_all['param_typ'].replace(['sedan/coupe'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['coupe/cabrio'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['minivan/city-car'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['compact/minivan'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['city-car/suv'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['coupe/cabrio/minivan'],'others')
df_all['param_typ'] = df_all['param_typ'].replace(['compact/sedan'],'others')

In [22]:
df_all['param_moc'] = df_all.param_moc.str.replace('KM?' , '')
df_all['param_moc'] = df_all.param_moc.str.replace(' ?' , '')
df_all['param_moc_num'] = pd.to_numeric(df_all['param_moc'])
cut_labels = ['Low', 'LowMid', 'Mid', 'LowHigh', 'MidHigh', 'High']
cut_bins = [0, 100, 150, 200, 300, 500, 1000000]
df_all['moc_bin'] = pd.cut(df_all['param_moc_num'], bins=cut_bins, labels=cut_labels)

In [23]:
df_all['param_pojemność_skokowa'] = df_all['param_pojemność-skokowa']
df_all['param_pojemność_skokowa']= df_all.param_pojemność_skokowa.str.replace('cm3?' , '')
df_all['param_pojemność_skokowa']= df_all.param_pojemność_skokowa.str.replace(' ?' , '')
df_all['pojemność_skokowa_num'] = pd.to_numeric(df_all['param_pojemność_skokowa'])
cut_labels = ['Low', 'LowMid', 'Mid', 'LowHigh', 'High']
cut_bins = [0, 1000, 1500, 2000, 3000, 9000]
df_all['pojemność_skokowa_bin'] = pd.cut(df_all['pojemność_skokowa_num'], bins=cut_bins, labels=cut_labels)

In [24]:
df_all['param_przebieg']= df_all.param_przebieg.str.replace('km?' , '')
df_all['param_przebieg']= df_all.param_przebieg.str.replace(' ?' , '')
df_all['przebieg_num'] = pd.to_numeric(df_all['param_przebieg'])
cut_labels = ['Low', 'MidLow',  'Mid', 'MidHigh', 'High']
cut_bins = [0, 100000, 150000, 200000, 300000, 1000000]
df_all['przebieg_bin'] = pd.cut(df_all['przebieg_num'], bins=cut_bins, labels=cut_labels)

In [25]:
df_all['param_rok-produkcji'] = pd.to_numeric(df_all['param_rok-produkcji'])
df_all['param_rok-produkcji'] = df_all['param_rok-produkcji'].fillna(-1)
df_all['wiek'] = 2019 - df_all['param_rok-produkcji']
cut_labels = ['0-5', '5-10', '10-50', '50-100']
cut_bins = [0, 5, 10, 50, 100]
df_all['wiek_bin'] = pd.cut(df_all['wiek'], bins=cut_bins, labels=cut_labels)

In [26]:
df_one = pd.get_dummies(df_all['moc_bin'], prefix='moc_km').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['pojemność_skokowa_bin'], prefix='pojemność_skokowa').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['przebieg_bin'], prefix='przebieg').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['kraj_norm'], prefix='kraj').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['drzwi_norm'], prefix='drzwi').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['param_kolor'], prefix='kolor').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['wiek_bin'], prefix='wiek').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['param_rodzaj-paliwa'], prefix='rodzaj_paliwa').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['param_skrzynia-biegów'], prefix='skrzynia_biegów').head()
df_all = pd.concat([df_all, df_one], axis=1)

df_one = pd.get_dummies(df_all['param_typ'], prefix='typ').head()
df_all = pd.concat([df_all, df_one], axis=1)

In [27]:
cat_feats = df_all.select_dtypes(include=[np.object]).columns

for cat_feat in cat_feats:
    df_all['{0}_cat'.format(cat_feat)] = df_all[cat_feat].factorize()[0]

In [28]:
df_all['kraj_norm_cat'] = df_all['kraj_norm'].factorize()[0]
df_all['drzwi_norm_cat'] = df_all['drzwi_norm'].factorize()[0]
df_all['param_skrzynia-biegów_cat'] = df_all['param_skrzynia-biegów'].factorize()[0]
df_all['param_kolor_cat'] = df_all['param_kolor'].factorize()[0]
df_all['param_rodzaj-paliwa_cat'] = df_all['param_rodzaj-paliwa'].factorize()[0]
df_all['przebieg_bin_cat'] = df_all['przebieg_bin'].factorize()[0]
df_all['moc_bin_cat'] = df_all['moc_bin'].factorize()[0]
df_all['pojemność_skokowa_bin_cat'] = df_all['pojemność_skokowa_bin'].factorize()[0]
df_all['param_typ_cat'] = df_all['param_typ'].factorize()[0]
df_all['wiek_bin_cat'] = df_all['wiek_bin'].factorize()[0]

In [29]:
# Funkcja wyciągająca zbiór treningowy

In [30]:
def get_train(df):
    return df.loc[df['price_value'] >= 0]

In [31]:
# Funkcja wyciągająca zbiór testowy

In [32]:
def get_test(df):
    return df.loc[df['price_value'] < 0]

In [33]:
# Funkcja wygiącająca cechy brane do modelu

In [34]:
def get_feats(df):
    feats = df.select_dtypes(include=[np.int, np.int8, np.int16, np.int32, np.float, np.float32]).columns
    black_list = ['car_id','price_value', 'price_log', 'feature_czujniki-parkowania-przednie_cat', 'feature_poduszka-powietrzna-chroniąca-kolana_cat', 
 'feature_kurtyny-powietrzne_cat', 'feature_klimatyzacja-dwustrefowa_cat', 'feature_światła-led_cat', 'feature_czujnik-zmierzchu_cat',
 'feature_elektrycznie-ustawiane-lusterka_cat', 'feature_asr-(kontrola-trakcji)_cat', 'feature_poduszka-powietrzna-kierowcy_cat',
 'feature_cd_cat', 'feature_elektryczne-szyby-przednie_cat', 'feature_poduszka-powietrzna-pasażera_cat', 'feature_system-start-stop_cat',
 'feature_światła-do-jazdy-dziennej_cat', 'feature_komputer-pokładowy_cat', 'feature_elektryczne-szyby-tylne_cat', 'feature_klimatyzacja-manualna_cat',
 'feature_tapicerka-welurowa_cat', 'feature_czujnik-deszczu_cat', 'feature_światła-przeciwmgielne_cat', 'feature_ogrzewanie-postojowe_cat',
 'feature_radio-niefabryczne_cat', 'feature_regulowane-zawieszenie_cat', 'feature_ogranicznik-prędkości_cat', 'feature_zmieniarka-cd_cat',
 'feature_szyberdach_cat', 'feature_isofix_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_alufelgi_cat', 'feature_bluetooth_cat',
 'feature_nawigacja-gps_cat', 'feature_asystent-parkowania_cat', 'feature_wspomaganie-kierownicy_cat', 'feature_podgrzewana-przednia-szyba_cat',
 'feature_przyciemniane-szyby_cat', 'feature_elektrycznie-ustawiane-fotele_cat', 'feature_klimatyzacja-czterostrefowa_cat', 'feature_tuner-tv_cat',
 'feature_poduszki-boczne-przednie_cat', 'feature_tempomat-aktywny_cat', 'feature_klimatyzacja-automatyczna_cat', 'feature_poduszki-boczne-tylne_cat',
 'feature_radio-fabryczne_cat', 'feature_odtwarzacz-dvd_cat', 'feature_czujniki-parkowania-tylne_cat', 'feature_łopatki-zmiany-biegów_cat',
 'feature_centralny-zamek_cat', 'feature_dach-panoramiczny_cat', 'feature_kamera-cofania_cat', 'feature_immobilizer_cat', 'feature_czujnik-martwego-pola_cat',
 'feature_gniazdo-sd_cat', 'feature_podgrzewane-lusterka-boczne_cat', 'feature_podgrzewane-przednie-siedzenia_cat', 'feature_hak_cat',
 'feature_abs_cat', 'feature_wielofunkcyjna-kierownica_cat', 'feature_światła-xenonowe_cat', 'feature_gniazdo-usb_cat', 'feature_tapicerka-skórzana_cat',
 'feature_gniazdo-aux_cat', 'feature_esp-(stabilizacja-toru-jazdy)_cat', 'feature_alarm_cat', 'feature_podgrzewane-tylne-siedzenia_cat',
 'feature_mp3_cat', 'feature_tempomat_cat', 'feature_hud-(wyświetlacz-przezierny)_cat', 'feature_elektrochromatyczne-lusterko-wsteczne_cat',
 'feature_relingi-dachowe_cat', 'feature_elektrochromatyczne-lusterka-boczne_cat', 'param_moc_num', 'pojemność_skokowa_num', 'przebieg_num',
 'wiek', 'param_engine-code_cat', 'param_first-registration_cat', 'param_country-of-origin_cat', 'param_gearbox_cat', 'param_faktura-vat_cat', 'param_vin_cat',
 'param_service-record_cat', 'param_metallic_cat', 'param_leasing-concession_cat', 'param_color_cat', 'param_financing-option_cat', 'param_original-owner_cat',
 'param_vat-marża_cat', 'param_co2-emissions_cat', 'param_leasing_cat', 'param_mileage_cat', 'param_wartość-wykupu_cat', 'param_vat-discount_cat',
 'param_no-accident_cat', 'param_liczba-pozostałych-rat_cat', 'param_particle-filter_cat', 'param_oferta-od_cat', 'param_make_cat', 'param_engine-power_cat',
 'param_year_cat', 'param_możliwość-finansowania_cat', 'param_registered-in-poland_cat', 'param_transmission_cat', 'param_miesięczna-rata_cat',
 'param_kod-silnika_cat', 'param_version_cat', 'param_opłata-początkowa_cat', 'param_fuel-type_cat', 'param_vat-free_cat', 'param_acrylic_cat', 'param_damaged_cat',
 'param_door-count_cat', 'param_tuning_cat', 'param_filtr-cząstek-stałych_cat', 'param_nr-of-seats_cat', 'param_engine-capacity_cat', 'param_pearl_cat',
 'param_emisja-co2_cat', 'param_body-type_cat', 'param_monthly-payment-value_cat', 'kraj_-1', 'kraj_Austria', 'kraj_Belgia', 'kraj_Francja', 'kraj_Holandia',
 'kraj_Inny', 'kraj_Niemcy', 'kraj_Polska', 'kraj_Stany Zjednoczone', 'kraj_Szwajcaria', 'kraj_Wielka Brytania', 'kraj_Włochy', 'drzwi_-1', 'drzwi_2', 'drzwi_3',
 'drzwi_4', 'drzwi_5', 'drzwi_Other', 'kolor_-1', 'kolor_Beżowy', 'kolor_Biały', 'kolor_Bordowy', 'kolor_Brązowy', 'kolor_Czarny', 'kolor_Czerwony', 'kolor_Fioletowy',
 'kolor_Inny kolor', 'kolor_Niebieski', 'kolor_Srebrny', 'kolor_Szary', 'kolor_Zielony', 'kolor_Złoty', 'kolor_Żółty', 'rodzaj_paliwa_-1', 'rodzaj_paliwa_Benzyna',
 'rodzaj_paliwa_Benzyna+CNG', 'rodzaj_paliwa_Benzyna+LPG', 'rodzaj_paliwa_Diesel', 'rodzaj_paliwa_Elektryczny', 'rodzaj_paliwa_Etanol', 'rodzaj_paliwa_Hybryda', 
 'rodzaj_paliwa_Wodór', 'skrzynia_biegów_-1', 'skrzynia_biegów_Automatyczna bezstopniowa (CVT)', 'skrzynia_biegów_Automatyczna dwusprzęgłowa (DCT, DSG)',
 'skrzynia_biegów_Automatyczna hydrauliczna (klasyczna)', 'skrzynia_biegów_Manualna', 'skrzynia_biegów_Półautomatyczna (ASG, Tiptronic)',
 'typ_-1', 'typ_Auta małe', 'typ_Auta miejskie', 'typ_Coupe', 'typ_Kabriolet', 'typ_Kombi', 'typ_Kompakt', 'typ_Minivan', 'typ_SUV', 'typ_Sedan',
 'typ_others']
    return [x for x in feats if x not in black_list]

In [35]:
# Funkcja wyciągająca X i y do trenowania modeli

In [36]:
def get_X(df, feat_fn=get_feats):
    return df[feat_fn(df)].values

def get_y(df, target_var='price_value'):
    return df[target_var].values

In [37]:
# Funkcja wczytująca model

In [38]:
def get_models():
    return [ ('dummy_median', DummyRegressor(strategy='median')) ]

In [39]:
# Funkcja do testowania modeli i cech, rysowania krzywej uczenia oraz trenowania/predykcji końcowej. 

In [40]:
def run_cv(model, X, y, folds=4, target_log=False,cv_type=KFold, success_metric=mean_absolute_error):
    cv = cv_type(n_splits=folds)
    
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        if target_log:
            y_train = np.log(y_train)
        
        print(f'Fitting model (fold: {len(scores) + 1})')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        if target_log:
            y_pred = np.exp(y_pred)
            y_pred[y_pred < 0] = 0 #czasem może być wartość ujemna

        score = success_metric(y_test, y_pred)
        scores.append( score )
        
    return np.mean(scores), np.std(scores)

In [41]:
def plot_learning_curve(model, title, X, y, ylim=None, cv=None, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5), target_log=False, success_metric=mean_absolute_error):
    plt.figure(figsize=(12,8))
    plt.title(title)
    if ylim is not None: plt.ylim(*ylim)

    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    if target_log:
        y = np.log(y)
    
    def my_scorer(model, X, y):
        y_pred = model.predict(X)
        
        if target_log:
            y = np.exp(y)
            y_pred = np.exp(y_pred)
            y_pred[ y_pred<0 ] = 0
        
        return success_metric(y, y_pred)

        
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=my_scorer)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,  test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    return plt


def run(df, plot_lc=False, folds=3, ylim=(0, 10000), target_log=False, feats=get_feats, success_metric=mean_absolute_error):
    X, y = get_X(df, feats), get_y(df)

    for model_name, model in get_models():
        score_mean, score_std = run_cv(model, X, y, folds=folds, target_log=target_log, success_metric=success_metric)
        print("[{0}]: {1} +/-{2}".format(model_name, score_mean, score_std))
        sys.stdout.flush() #wypisujemy wynik natychmiast, bez buforowania

        if False == plot_lc: continue
        plt = plot_learning_curve(model, model_name, X, y, ylim=ylim, cv=folds, target_log=target_log, n_jobs=3, success_metric=success_metric)
        plt.show()

In [42]:
def train_and_predict(train_new, test_new, model, target_log=False, feats=get_feats):
    
    X_train, X_test = get_X(train_new, feats), get_X(test_new, feats)
    y_train = get_y(train_new)
    
    print(sorted(list(feats(train_new))))
    
    if target_log:
        y_train = np.log(y_train)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
        
    if target_log:
        y_pred = np.exp(y_pred)

    global_min = train.price_value.min()
    y_pred[y_pred < global_min] = global_min

    test_new['price_value'] = y_pred
    
    return test_new[['car_id', 'price_value']] 

In [43]:
# Trenowanie modeli: CatBoost do szybkiego sprawdzenia hipotez, XGBoost do ostatecznego trenowania na całym zbiorze danych.

In [44]:
train_new = get_train(df_all)

In [45]:
test_new = get_test(df_all)

In [46]:
train_new['price_log'] = np.log(train_new['price_value']) # logarytmowanie targetu w celu poprawienia rozkładu

/home/conda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
feats = get_feats(df_all)

In [48]:
X_train, y_train = get_X(train_new), get_y(train_new)

In [49]:
cb_params = dict(iterations=2000, verbose=False)

def get_models():
    return [('cb', CatBoostRegressor(**cb_params))]

run(train_new, plot_lc=False, ylim=(0, 10000), folds=3, target_log=True, feats=get_feats)

Fitting model (fold: 1)
Fitting model (fold: 2)
Fitting model (fold: 3)
[cb]: 7195.535964478507 +/-80.91716139858057


In [50]:
xgb_params = {
    'max_depth': 20,
    'n_estimators': 10000,
    'learning_rate': 0.01,
    'random_state': 0,
    'subsample': 0.9,
    'colsample_bytree': 0.5,
}

In [51]:
def get_models():
    return [('xgb', xgb.XGBRegressor(**xgb_params))]

run(train_new, plot_lc=False, ylim=(0, 10000), folds=3, target_log=True, feats=get_feats)

Fitting model (fold: 1)
Fitting model (fold: 2)
Fitting model (fold: 3)
[xgb]: 6516.479719016319 +/-101.55447135370515


In [52]:
model = xgb.XGBRegressor(**xgb_params)

In [53]:
final = train_and_predict(train_new, test_new, model, target_log=True, feats=get_feats)

['amenities', 'drzwi_norm_cat', 'kraj_norm_cat', 'moc_bin_cat', 'moc_km_High', 'moc_km_Low', 'moc_km_LowHigh', 'moc_km_LowMid', 'moc_km_Mid', 'moc_km_MidHigh', 'param_akryl-(niemetalizowany)_cat', 'param_bezwypadkowy_cat', 'param_homologacja-ciężarowa_cat', 'param_kategoria_cat', 'param_kierownica-po-prawej-(anglik)_cat', 'param_kolor_cat', 'param_kraj-pochodzenia_cat', 'param_liczba-drzwi_cat', 'param_liczba-miejsc_cat', 'param_marka-pojazdu_cat', 'param_matowy_cat', 'param_metalik_cat', 'param_moc_cat', 'param_model-pojazdu_cat', 'param_model_cat', 'param_napęd_cat', 'param_perłowy_cat', 'param_pierwsza-rejestracja_cat', 'param_pierwszy-właściciel_cat', 'param_pojemność-skokowa_cat', 'param_pojemność_skokowa_cat', 'param_przebieg_cat', 'param_rodzaj-paliwa_cat', 'param_rok-produkcji', 'param_serwisowany-w-aso_cat', 'param_skrzynia-biegów_cat', 'param_stan_cat', 'param_typ_cat', 'param_uszkodzony_cat', 'param_wersja_cat', 'param_zarejestrowany-jako-zabytek_cat', 'param_zarejestrowany-

/home/conda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
final.to_csv('final_car_price.csv', index=False)

In [55]:
# Wynik na Kaggle ostatecznie to 6043.39861 więc całkiem nieźle, biorą pod uwagę fakt usunięcia pierwszych sześciu kolumn,
# z których można wyciągnąć jeszcze trochę informacji, które mogą pomóć w douczeniu modelu.